In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm

np.random.seed(1024)
VERBOSE = False

## 1. IO


In [2]:
NETSCALE = 16

In [3]:
network = pd.read_csv("%s_motivate_topology.csv"%NETSCALE)
task = pd.read_csv("%s_0_motivate_task.csv"%NETSCALE)

In [4]:
## Shortest path
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for _next in set(np.reshape(np.argwhere(graph[vertex] > 0),  -1)) - set(path):
            if _next == goal:
                yield path + [_next]
            else:
                queue.append((_next, path + [_next]))

## 2. Simulation

### Constant Values

In [5]:
## Network setting
NUM_ES = NETSCALE
NUM_SW = NETSCALE
NUM_NODE = NUM_ES + NUM_SW
NUM_PORT = 4
NUM_QUEUES = 8

PROC = 1_000

## Task setting
NUM_FLOW = len(task)

## Global setting
GRANULARITY = 100

### initial variables

In [6]:
'''
Simulation for CBS:

paramters: SLOPE = {link: [[idle, send]0, ...., [idle, send]N]}
status: 
        CREDIT = {link: [credit0,..., creditN]}
        ACTION = {link: [0, 1, 2,..., N]}
            0 -> idle [recover]
            1 -> waitting [increase]
            2 -> transmitting [decrease]
        

'''

CREDIT= {}
SLOPE = {}
ACTION = {}


period = list(task['period'])
size = list(task['size'])
deadline = list(task['deadline'])
MTU = np.lcm.reduce(period)

In [7]:
net = np.zeros(shape = (NUM_NODE, NUM_NODE))
for _, row in network.iterrows():
    link = eval(row['Link'])
    net[eval(row['Link'])[0], eval(row['Link'])[1]] = 1
    CREDIT[link] = [0 for i in range(8)]
    ACTION[link] = [0 for i in range(8)]
    SLOPE[link] = [[0,0] for i in range(8)]

In [8]:
route = {}
_route = {}

for i, row in task.iterrows():
    route[i] = eval(str(next(bfs_paths(net, int(row['start']), int(row['end'])))))
    _route[i] = {x: int(eval(str(route[i]))[k+1]) for k, x in enumerate(eval(str(route[i]))[:-1])} 

In [9]:
queue = {}
for i in range(len(task)):
    queue.setdefault(i, {})
    for link in _route[i].items():
        queue[i][link] = np.random.randint(0, 8)
        q = queue[i][link]
        SLOPE[link][q][0] += (size[i] * 8 / period[i])

for i in queue:
    for link in queue[i]:
        q = queue[i][link]
        SLOPE[link][q][1] = 1 - SLOPE[link][q][0]

In [10]:
# For multicast
# src = {}
# dst = {}
# for i, row in task.iterrows():
#     src[i] = row['start']
#     dst[i] = set(eval(row['end']))

## Shortest path
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for _next in set(np.reshape(np.argwhere(graph[vertex] > 0),  -1)) - set(path):
            if _next == goal:
                yield path + [_next]
            else:
                queue.append((_next, path + [_next]))

In [11]:
t_release = [0] * NUM_FLOW

### Calculate latency and jitter

Calaucate the release time of each flow.

In [12]:
log = [[[], []] for i in range(NUM_FLOW)]

In [13]:
'''
Simulation for CBS:

paramters: SLOPE = {link: [[idle, send]0, ...., [idle, send]N]}
status: 
        CREDIT = {link: [credit0,..., creditN]}
        ACTION = {link: [0, 1, 2]}
            0 -> idle [recover]
            1 -> waitting [increase]
            2 -> transmitting [decrease]
        

'''

log = [[[], []] for i in range(NUM_FLOW)]
egress_q = {link: [[] for i in range(NUM_QUEUES)] for link, _ in CREDIT.items()}
mutex_lock = {link: 0 for link, _ in CREDIT.items()}
_pool = {link: [] for link, _ in CREDIT.items()}

for t in tqdm(range(0, MTU * 1, GRANULARITY)):
    if t % MTU == 0:
        print("[Net Cycle] ------ %d ----------"%(t // MTU))
        
    ## Release task
    for flow in range(NUM_FLOW):
        if t % period[flow] == 0:
            link = (route[flow][0], route[flow][1])
            egress_q[link][queue[flow][link]].append(flow)  
            log[flow][0].append(t)
            if VERBOSE:
                print("Flow %d: Released at %d"%(flow,t))
            
    ## Timer - TODO: Replace by heap
    for link, vec in _pool.items():
        _new_vec = []
        for ct, flow in vec:
            if t >= ct:
                if link[-1] == route[flow][-1]: 
                    log[flow][1].append(t)
                    if VERBOSE:
                        print("Flow %d: Received at %d"%(flow,t))
                    continue
                # elif link[0] == route[flow][0]:
                #     log[flow][0].append(t)
                #     if VERBOSE:
                #         print("Flow %d: Released at %d"%(flow,t))
                new_link = (link[-1], _route[flow][link[-1]])
                egress_q[new_link][queue[flow][new_link]].append(flow)
            else:
                _new_vec.append((ct, flow))
        _pool[link] = _new_vec
    
    # Qbv
    for link in CREDIT:
        ## Change the credit based on ACTION
        
        for q, action in enumerate(ACTION[link]):
            if action == 1:
                CREDIT[link][q] += SLOPE[link][q][0] * GRANULARITY
            elif action == 2:
                CREDIT[link][q] -= SLOPE[link][q][1] * GRANULARITY
            elif action == 0 and CREDIT[link][q] != 0:
                CREDIT[link][q] = CREDIT[link][q] + SLOPE[link][q][0] * GRANULARITY if CREDIT[link][q] < 0 else \
                                  CREDIT[link][q] - SLOPE[link][q][1] * GRANULARITY
        
        ## Enumberate every queue and check priority:
        _in_trans = t < mutex_lock[link]
        for q, credit in reversed(list(enumerate(CREDIT[link]))):
            if egress_q[link][q]:
                if CREDIT[link][q] >= 0 and not _in_trans:
                    trans_delay = size[egress_q[link][q][0]] * 8
                    out = egress_q[link][q].pop(0)
                    _pool[link].append((t + trans_delay + PROC, out))
                    mutex_lock[link] = t + trans_delay
                    ACTION[link][q] = 2 ## In trans
                    _in_trans = True
                else:
                    ACTION[link][q] = 1 ## Waitting
            else: ## Empty
                ACTION[link][q] = 0

  0%|          | 0/6000000 [00:00<?, ?it/s]

[Net Cycle] ------ 0 ----------


100%|██████████| 6000000/6000000 [34:18<00:00, 2914.42it/s]


### Define Schedule

In [14]:
for flow in log:
    print([flow[1][i] - flow[0][i] for i in range(len(flow[1]))])

[871300, 992100, 771700, 582400, 1002500, 765700, 590000, 995100, 774700, 578600, 1117300, 819300, 591000, 999500, 761300, 581400, 992100, 771700, 582400, 1002500, 790300, 590000, 1110000, 875900, 586200, 989100, 768700, 380000, 999500, 761300, 589000, 992100, 771700, 582400, 1112200, 1076400, 594000, 995100, 774700, 578600, 989100, 768700, 380000, 999500, 761300, 581400, 1110000, 1076600, 688400, 1002500, 790300, 590000, 995100, 774700, 578600, 989100, 768700, 380000, 1109300, 1012900]
[664800, 677600, 658500, 633300, 653100, 658900, 657800, 656600, 655400, 654200, 646100, 665900, 657700, 656500, 690300, 647200, 660200, 637900, 664600, 670500, 655300, 640200, 694900, 644800, 664600, 656400, 676200, 647100, 645900, 651700, 678500, 649400, 641200, 654000, 659800, 644700, 671500, 656400, 662100, 667900, 652800, 644600, 643500, 649300, 669100, 653900, 666700, 679500, 643400, 642200, 662000, 667800, 659700, 651500, 657300, 656100, 648000, 647000, 673600, 651500, 650300, 663100, 640900, 653

In [15]:
with open("log_%s.txt"%NETSCALE, 'w') as f:
    f.write(str(log))

In [16]:
SLOPE

{(0, 1): [[0, 0],
  [0.0012333333333333335, 0.9987666666666667],
  [0, 0],
  [0, 0],
  [0, 0],
  [8.333333333333333e-05, 0.9999166666666667],
  [0, 0],
  [8e-05, 0.99992]],
 (16, 0): [[0, 0],
  [0, 0],
  [0, 0],
  [0.00048333333333333334, 0.9995166666666667],
  [0, 0],
  [0.0008333333333333334, 0.9991666666666666],
  [0, 0],
  [8e-05, 0.99992]],
 (0, 16): [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]],
 (1, 0): [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]],
 (1, 2): [[0.0008333333333333334, 0.9991666666666666],
  [0.0004, 0.9996],
  [0, 0],
  [0, 0],
  [0, 0],
  [0.00016333333333333334, 0.9998366666666667],
  [0.0025800000000000003, 0.99742],
  [0.007859999999999999, 0.99214]],
 (17, 1): [[8e-05, 0.99992],
  [0.01036, 0.98964],
  [0, 0],
  [0, 0],
  [0, 0],
  [0, 0],
  [0, 0],
  [0, 0]],
 (1, 17): [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]],
 (2, 1): [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]],
 (2, 3): [[0.